In [27]:
import logging, os, sys
import numpy as np
import urllib.request
import shutil
import subprocess
import zipfile
import pickle
from pathlib import Path
import pandas as pd
from Pegasus.api import *

In [28]:
from util_workflow import download_data, unzip_flatten_data, return_corrupted_files, return_input_files
from util_workflow import add_input_wf_files, add_output_job1, add_output_job2, return_filenames_job2
from util_workflow import split_data_filenames, add_input_tune_model,create_tar_and_pkl, create_pkl

In [29]:
# --- Import Pegasus API ---
from Pegasus.api import *
logging.basicConfig(level=logging.DEBUG)
props = Properties()
props["dagman.retry"] = "2"
props["pegasus.transfer.arguments"] = "-m 1"
props.write()

In [30]:
dataset_link = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"
zip_data = "kagglecatsanddogs_3367a.zip"
directory_to_extract_to = "."

DOWNLOAD_DATA = False
DATASET_SIZE = 12
DATA_DIR = "dev_data/"
UTILS_DIR = "utils/"
DATA_SPLIT_FILE = "data_split_id_list.pickle"

arch_names = ["basicnet", "densenet121", "vgg16"]
CATS = "PetImages/Cat"
DOGS = "PetImages/Dog"
LABELS = {CATS: 0, DOGS: 1}
NUM_EPOCHS = 4
NUM_TRIALS = 3

if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)   
    
if DOWNLOAD_DATA == True:
    download_data(dataset_link)

In [31]:
# Avoid corrupted files for now
corrupted_files = return_corrupted_files("corrupted_files.txt")

# Get names of image files that will serve as inputs to the workflow
input_file_names = return_input_files(corrupted_files, DATASET_SIZE, DATA_DIR, LABELS)

In [32]:
def add_input_wf_files(input_file_names, DATA_DIR,rc):
    """ Inputs to data_preprocessing1.py
    """
    all_input_files  = []
    for filename in input_file_names:
        dst = DATA_DIR + filename
        print(os.path.join(os.getcwd(), dst))
        all_input_files.append(File(filename))
        rc.add_replica("local", filename,  os.path.join(os.getcwd(), dst)) 
    return all_input_files

In [33]:
rc = ReplicaCatalog()

input_preprocess1 = add_input_wf_files(input_file_names, DATA_DIR,rc)

/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_11287.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_9395.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_11992.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_12249.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_5786.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Cat_2396.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_11287.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_9395.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_11992.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_12249.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_5786.jpg
/home/scitech/shared-data/pegasus-catdog-wf-master/dev_data/Dog_2396.jpg


In [34]:
input_preprocess2 = add_output_job1(input_file_names)

In [35]:
rc.write()

In [36]:
# Create and add our transformations to the TransformationCatalog.
tc = TransformationCatalog()

# Data preprocessing part 1 
preprocess_tc1 = Transformation(
                "preprocess1",
                site="local",
                pfn = str(Path(".").parent.resolve() / "bin/data_preprocessing1.py"), 
                is_stageable= True
            )

In [37]:
tc.add_transformations(preprocess_tc1) #, preprocess2,data_split,tune_model_vgg16,choose_best_model)
tc.write()

In [38]:
# Set infer_dependencies=True so that they are inferred based on job input and output file usage.
wf = Workflow("catVsdog-test-wf", infer_dependencies=True)

# Create Jobs. These objects store just that. The transformation (executable) used by the job.
#The arguments passed to the executable. The input files used and the output files produced. 


job_preprocess_1 = Job(preprocess_tc1)\
                    .add_inputs(*input_preprocess1)\
                    .add_outputs(*input_preprocess2)


In [39]:
wf.add_jobs(
    job_preprocess_1#,
#    job_preprocess_2,
#    job_data_split#,
#    job_tune_model_vgg16,
#    job_choose_best_model
)


In [40]:
try:
    wf.plan(submit=True)\
    .wait()\
    .analyze()\
    .statistics()
except PegasusClientError as e:
    print(e.output)


################
# pegasus-plan #
################
2021.02.08 06:08:54.361 UTC:
2021.02.08 06:08:54.366 UTC:   -----------------------------------------------------------------------
2021.02.08 06:08:54.371 UTC:   File for submitting this DAG to HTCondor           : catVsdog-test-wf-0.dag.condor.sub
2021.02.08 06:08:54.376 UTC:   Log of DAGMan debugging messages                 : catVsdog-test-wf-0.dag.dagman.out
2021.02.08 06:08:54.381 UTC:   Log of HTCondor library output                     : catVsdog-test-wf-0.dag.lib.out
2021.02.08 06:08:54.387 UTC:   Log of HTCondor library error messages             : catVsdog-test-wf-0.dag.lib.err
2021.02.08 06:08:54.392 UTC:   Log of the life of condor_dagman itself          : catVsdog-test-wf-0.dag.dagman.log
2021.02.08 06:08:54.397 UTC:
2021.02.08 06:08:54.402 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with:
2021.02.08 06:08:54.412 UTC:   -----------------------------------------------------------------------


[####################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)



####################
# pegasus-analyzer #
####################
Your database is compatible with Pegasus version: 5.1.0dev

************************************Summary*************************************

Submit Directory   : /home/scitech/shared-data/pegasus-catdog-wf-master/scitech/pegasus/catVsdog-test-wf/run0008
Total jobs         :      9 (100.00%)
# jobs succeeded   :      9 (100.00%)
# jobs failed      :      0 (0.00%)
# jobs held        :      0 (0.00%)
# jobs unsubmitted :      0 (0.00%)


######################
# pegasus-statistics #
######################
Your database is compatible with Pegasus version: 5.1.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, job